In [1]:
#Cargar librerias necesarias
import pandas as pd
import os

In [2]:
# Carga el archivo Excel para obtener los nombres de las estaciones
df_estaciones = pd.read_excel("datos_ideam/nombre_estaciones/estaciones_todas.xlsx")  # Cambia esto por la ubicación real de tu archivo de Excel
estaciones_a_filtrar = df_estaciones.iloc[:, 0].tolist()

In [3]:
# Lista para almacenar los DataFrames
dfs = []

# Iterar solo sobre los archivos que corresponden a las estaciones en la lista
for estacion in estaciones_a_filtrar:
    filename = f"datos_ideam/pp_ideam_crudos/{estacion}.data"  # Asegúrate de que este formato coincide con tus archivos
    # Verifica si el archivo existe
    if os.path.exists(filename):
        # Leer el archivo en un DataFrame, especificando el delimitador
        df = pd.read_csv(filename, delimiter='|', parse_dates=['Fecha'])
        # Utilizar la columna "Fecha" como índice
        df.set_index('Fecha', inplace=True)
        # Cambiar el nombre de la columna de precipitación al nombre del archivo (sin extensión)
        df.rename(columns={"Valor": estacion}, inplace=True)
        # Añadir el DataFrame a la lista
        dfs.append(df)
    else:
        print(f"El archivo para la estación {estacion} no se encontró.")

In [4]:
# Concatenar todos los DataFrames
result = pd.concat(dfs, axis=1)
# Ordenar el DataFrame por la columna 'Fecha'
result.sort_index(inplace=True)
# Cargar el archivo Excel con los nombres de las estaciones
df_nombres = pd.read_excel("datos_ideam/nombre_estaciones/nombre_estaciones.xlsx")
# Crear un diccionario para mapear los códigos a los nombres
mapa_nombres = pd.Series(df_nombres.Nombre.values, index=df_nombres.Codigo).to_dict()
# Renombrar las columnas en el DataFrame 'result'
result.rename(columns=mapa_nombres, inplace=True)
# Guardar el DataFrame 'result' en un nuevo archivo Excel
result.to_excel("datos_ideam/resultados_concatenados.xlsx")